📎 [Haz clic aquí para acceder a mi notebook en Google Drive](https://drive.google.com/file/d/19lu3Mf7nVyoaaFR8iW-Ycg5TpDOeqsbh/view?usp=drive_link)


📎 [Haz clic aquí para ver la presentación en PDF](https://drive.google.com/file/d/1HOZ1TFqna5pYKXfqWaC64biZDbL861C8/view?usp=drive_link)


📊 [Haz clic aquí para ver el dashboard en Tableau Public](https://public.tableau.com/app/profile/dante.dupeyron/viz/DashboarddeAnlisisdeLibrosDigitales/Dashboard1)


# Proyecto Final de SQL — Análisis de Libros

Durante la pandemia del coronavirus, la vida cotidiana cambió drásticamente. Muchas personas dejaron de salir y comenzaron a pasar más tiempo en casa, lo cual incrementó el interés por la lectura. Esto atrajo la atención de startups que decidieron lanzar nuevas aplicaciones para lectores.

En este proyecto, analizaremos una base de datos de uno de estos servicios. Los datos incluyen información sobre libros, autores, editoriales, reseñas y calificaciones de usuarios. El objetivo es explorar estos datos y extraer conclusiones clave que ayuden a generar una propuesta de valor para un nuevo producto.

---

## Objetivos del estudio

1. Conectarnos correctamente a la base de datos.
2. Explorar el contenido de cada tabla para entender su estructura.
3. Responder las siguientes preguntas de negocio con SQL:
   - ¿Cuántos libros fueron publicados después del 1 de enero de 2000?
   - ¿Cuál es el número de reseñas y la calificación promedio por libro?
   - ¿Qué editorial publicó más libros con más de 50 páginas?
   - ¿Qué autor tiene la calificación promedio más alta considerando solo libros con al menos 50 calificaciones?
   - ¿Cuál es el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros?

---

## Descripción de las tablas

### books
- `book_id`: identificación del libro  
- `author_id`: identificación del autor o autora  
- `title`: título del libro  
- `num_pages`: número de páginas  
- `publication_date`: fecha de publicación  
- `publisher_id`: identificación de la editorial  

### authors
- `author_id`: identificación del autor o autora  
- `author`: nombre del autor o autora  

### publishers
- `publisher_id`: identificación de la editorial  
- `publisher`: nombre de la editorial  

### ratings
- `rating_id`: identificación de la calificación  
- `book_id`: identificación del libro  
- `username`: nombre del usuario  
- `rating`: calificación otorgada (entero)  

### reviews
- `review_id`: identificación de la reseña  
- `book_id`: identificación del libro  
- `username`: nombre del usuario  
- `text`: contenido de la reseña

---

## Herramientas utilizadas

- PostgreSQL para consultas SQL
- SQLAlchemy para conexión
- pandas para visualización de resultados

---


In [5]:
# Importar librerias 
import pandas as pd
from sqlalchemy import create_engine

In [6]:
# Configuración para conectarse a la base de datos
db_config = {
    'user': 'practicum_student',  # nombre de usuario
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',  # contraseña
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',  # host
    'port': 5432,  # puerto de conexión
    'db': 'data-analyst-final-project-db'  # nombre de la base de datos
}

# Construimos la cadena de conexión
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

In [7]:
# Creamos la conexión con la base de datos
engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

# Exploración de las tablas

Antes de comenzar con las consultas, vamos a explorar los primeros registros de cada tabla para entender su contenido y estructura.

In [8]:
# Exploramos books
query = 'SELECT * FROM books LIMIT 5'
pd.read_sql(query, con=engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [9]:
# Exploramos authors
query = 'SELECT * FROM authors LIMIT 5'
pd.read_sql(query, con=engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [10]:
# Exploramos publishers
query = 'SELECT * FROM publishers LIMIT 5'
pd.read_sql(query, con=engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [11]:
# Exploramos ratings
query = 'SELECT * FROM ratings LIMIT 5'
pd.read_sql(query, con=engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [12]:
# Exploramos reviews
query = 'SELECT * FROM reviews LIMIT 5'
pd.read_sql(query, con=engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# ¿Cuántos libros fueron publicados después del 1 de enero de 2000?

In [13]:
# Total de libros en la tabla
query = '''
SELECT COUNT(*) AS total_books 
FROM books
'''
pd.read_sql(query, con=engine)


,total_books
0,1000


In [14]:
# Contamos los libros publicados después del 1 de enero de 2000
query = '''
SELECT COUNT(*) AS books_published_after_2000
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.read_sql(query, con=engine)

,books_published_after_2000
0,819


Se identificaron **819 libros publicados después del 1 de enero del año 2000**. 

# ¿Cuál es el número de reseñas de usuarios y la calificación promedio para cada libro?

In [15]:
query = '''
SELECT 
    b.book_id,
    b.title,
    COUNT(rt.rating_id) AS num_ratings,
    ROUND(AVG(rt.rating), 2) AS avg_rating
FROM books b
JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
HAVING COUNT(rt.rating_id) >= 50
ORDER BY avg_rating DESC
'''
pd.read_sql(query, con=engine)


,book_id,title,num_ratings,avg_rating
0,302,Harry Potter and the Prisoner of Azkaban (Harr...,82,4.41
1,722,The Fellowship of the Ring (The Lord of the Ri...,74,4.39
2,299,Harry Potter and the Chamber of Secrets (Harry...,80,4.29
3,656,The Book Thief,53,4.26
4,300,Harry Potter and the Half-Blood Prince (Harry ...,73,4.25
5,301,Harry Potter and the Order of the Phoenix (Har...,75,4.19
6,399,Little Women,52,4.19
7,750,The Hobbit or There and Back Again,88,4.13
8,779,The Lightning Thief (Percy Jackson and the Oly...,62,4.08
9,405,Lord of the Flies,71,3.90


Los libros con más calificaciones y mejores promedios incluyen títulos ampliamente reconocidos como *Harry Potter*, *El Señor de los Anillos* y *The Book Thief*. Estos resultados confirman que los títulos más populares también tienden a tener una alta valoración, lo cual puede guiar futuras recomendaciones editoriales.

> Nota: Aunque la pregunta no pedía un filtro explícito, se incluyó `HAVING COUNT(rating_id) >= 50` para asegurar que los libros analizados tengan una base estadística sólida. Esto evita destacar libros con solo 1 o 2 calificaciones, que podrían distorsionar los resultados.

# ¿Qué editorial ha publicado el mayor número de libros con más de 50 páginas?

In [16]:
# Editorial con más libros de más de 50 páginas
query = '''
SELECT 
    p.publisher,
    COUNT(b.book_id) AS books_over_50_pages
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY books_over_50_pages DESC
LIMIT 5
'''
pd.read_sql(query, con=engine)

,publisher,books_over_50_pages
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


**Penguin Books** es la editorial con más libros de más de 50 páginas, con un total de **42 publicaciones** sustanciales. Esto indica que tiene un catálogo amplio y confiable.

# ¿Qué autor tiene la calificación promedio más alta, considerando solo libros con al menos 50 calificaciones?

In [17]:
query = '''
WITH rated_books AS (
    SELECT 
        book_id,
        COUNT(rating_id) AS rating_count,
        AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating_id) >= 50
)

SELECT 
    a.author_id,
    a.author,
    COUNT(rb.book_id) AS qualified_books,
    ROUND(AVG(rb.avg_rating), 2) AS author_avg_rating
FROM rated_books rb
JOIN books b ON rb.book_id = b.book_id
JOIN authors a ON b.author_id = a.author_id
GROUP BY a.author_id, a.author
ORDER BY author_avg_rating DESC
LIMIT 10
'''
pd.read_sql(query, con=engine)

,author_id,author,qualified_books,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4,4.28
1,240,J.R.R. Tolkien,2,4.26
2,402,Markus Zusak/Cao Xuân Việt Khương,1,4.26
3,376,Louisa May Alcott,1,4.19
4,498,Rick Riordan,1,4.08
5,621,William Golding,1,3.90
6,235,J.D. Salinger,1,3.83
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,1,3.79
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,1,3.79
9,372,Lois Lowry,1,3.75


El autor con la mejor calificación promedio, considerando únicamente libros con al menos 50 calificaciones, es **J.K. Rowling/Mary GrandPré** con un promedio de **4.28 en 4 libros**. Esto demuestra no solo popularidad, sino también calidad sostenida en múltiples obras.

# ¿Cuál es el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros?

In [18]:
# Promedio de reseñas de texto entre usuarios que calificaron más de 50 libros
query = '''
WITH active_users AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(book_id) > 50
),

reviews_count AS (
    SELECT r.username, COUNT(r.review_id) AS total_reviews
    FROM reviews r
    JOIN active_users au ON r.username = au.username
    GROUP BY r.username
)

SELECT ROUND(AVG(total_reviews), 2) AS avg_reviews_per_active_user
FROM reviews_count
'''
pd.read_sql(query, con=engine)

,avg_reviews_per_active_user
0,24.33


Los usuarios que han calificado más de 50 libros han escrito en promedio **24.33 reseñas de texto**. Esto refleja un alto nivel de participación activa en la plataforma.

# Conclusión Final

Este proyecto permitió analizar el comportamiento de usuarios, libros, autores y editoriales en una plataforma de lectura digital. Al aplicar filtros estadísticos, obtuvimos resultados claros y representativos.

Entre los hallazgos clave:
- La mayoría de los libros publicados son posteriores al año 2000.
- J.K. Rowling y Tolkien destacan por la calidad sostenida de sus obras.
- Penguin Books lidera en volumen de publicaciones sustanciales.
- Los usuarios más activos también generan contenido valioso a través de reseñas.

Este tipo de análisis es esencial para definir estrategias de producto, recomendaciones y alianzas dentro del sector editorial digital.
